In [1]:
import itertools

from triagerx.model.module_factory import ModelFactory
from triagerx.system.triagerx import TriagerX
from loguru import logger
import torch
import pandas as pd
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

tqdm.pandas()

In [2]:
torch.manual_seed(42)

In [3]:
target_components = ["comp:vm", "comp:jvmti", "comp:jclextensions", "comp:test", "comp:build", "comp:gc"]
target_components = sorted(target_components)

In [4]:
# # Define active user map
# vm_users = [
#     "pshipton",
#     "keithc-ca",
#     "gacholio",
#     "tajila",
#     "babsingh",
#     "JasonFengJ9",
#     "fengxue-IS",
#     "hangshao0",
#     "theresa.t.mammarella",
#     "ChengJin01",
#     "singh264",
#     "thallium",
#     "ThanHenderson",
# ]
# jvmti_users = ["gacholio", "tajila", "babsingh", "fengxue-IS"]
# jclextensions_users = ["JasonFengJ9", "pshipton", "keithc-ca"]
# test_users = ["LongyuZhang", "annaibm", "sophiaxu0424", "KapilPowar", "llxia"]
# build_users = ["adambrousseau", "mahdipub"]
# gc_users = ["dmitripivkine", "amicic", "kangyining", "LinHu2016"]

# # Putting them in dictionaries
# components = {
#     "comp:vm": vm_users,
#     "comp:jvmti": jvmti_users,
#     "comp:jclextensions": jclextensions_users,
#     "comp:test": test_users,
#     "comp:build": build_users,
#     "comp:gc": gc_users,
# }

# # expected_users = [user for user_list in components.values() for user in user_list]
# # df = df[df["owner"].isin(expected_users)]

# # df = df[df["component"].isin(target_components)]

In [38]:
# df_train, df_test = train_test_split(df, test_size=0.1, shuffle=False)
df_train = pd.read_csv("/home/mdafifal.mamun/notebooks/triagerX/data/openj9/last_contribution/openj9_train.csv")
df_test = pd.read_csv("/home/mdafifal.mamun/notebooks/triagerX/data/openj9/last_contribution/openj9_test.csv")

In [6]:
expected_users = set(df_train.owner.unique())

In [40]:
sample_threshold = 20
developers = df_train["owner"].value_counts()
filtered_developers = developers.index[developers >= sample_threshold]
df_train = df_train[df_train["owner"].isin(filtered_developers)]

train_owners = set(df_train["owner"])
test_owners = set(df_test["owner"])

unwanted = list(test_owners - train_owners)

df_test = df_test[~df_test["owner"].isin(unwanted)]

logger.info(f"Training data: {len(df_train)}, Validation data: {len(df_test)}")
logger.info(f"Number of developers: {len(df_train.owner.unique())}")

logger.info(f"Train dataset size: {len(df_train)}")
logger.info(f"Test dataset size: {len(df_test)}")

2024-06-20 22:56:08.347 | INFO     | __main__:<module>:13 - Training data: 3323, Validation data: 379
2024-06-20 22:56:08.349 | INFO     | __main__:<module>:14 - Number of developers: 50
2024-06-20 22:56:08.350 | INFO     | __main__:<module>:16 - Train dataset size: 3323
2024-06-20 22:56:08.351 | INFO     | __main__:<module>:17 - Test dataset size: 379


In [8]:
# # Generate label ids
lbl2idx = dict(zip(df_train['owner'], df_train['owner_id']))
idx2lbl = dict(zip(df_train['owner_id'], df_train['owner']))

# train_owners = sorted(train_owners)

# for idx, dev in enumerate(train_owners):
#     lbl2idx[dev] = idx
#     idx2lbl[idx] = dev

# df_train["owner_id"] = df_train["owner"].apply(lambda owner: lbl2idx[owner])
# df_test["owner_id"] = df_test["owner"].apply(lambda owner: lbl2idx[owner])

In [9]:
df_train = df_train[df_train["component"].notna()]
df_test = df_test[df_test["component"].notna()]

In [10]:
comp_id2label = {}
comp_lbl2id = {}

for i, comp in enumerate(target_components):
    comp_id2label[i] = comp
    comp_lbl2id[comp] = i

In [11]:
base_transformer_models = ["microsoft/deberta-base", "roberta-base"]

In [12]:
developer_model_weights = "/work/disa_lab/projects/triagerx/models/openj9/triagerx_ensemble_u3_50_classes_last_dev_seed42_last_epoch_39.pt"
component_model_weights = "/work/disa_lab/projects/triagerx/models/openj9/component_triagerx_u3_6_classes_seed42.pt"

In [13]:
logger.debug("Modeling network...")
dev_model = ModelFactory.get_model(
    model_key="triagerx",
    output_size=len(df_train.owner_id.unique()),
    unfrozen_layers=3,
    num_classifiers=3,
    base_models=base_transformer_models,
    dropout=0.2,
    max_tokens=256,
    label_map=idx2lbl,
)

2024-06-20 22:12:37.474 | DEBUG    | __main__:<module>:1 - Modeling network...
2024-06-20 22:12:37.475 | DEBUG    | triagerx.model.module_factory:get_model:54 - Instantiating model of class: <class 'triagerx.model.triagerx_dev_model.TriagerxDevModel'>
2024-06-20 22:12:37.477 | DEBUG    | triagerx.model.module_factory:get_model:65 - Including number of filters and classifiers
2024-06-20 22:12:37.478 | DEBUG    | triagerx.model.triagerx_dev_model:__init__:24 - Loading base transformer models: ['microsoft/deberta-base', 'roberta-base']
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
dev_model.load_state_dict(torch.load(developer_model_weights))

<All keys matched successfully>

In [15]:
comp_model = ModelFactory.get_model(
    model_key="triagerx",
    output_size=6,
    unfrozen_layers=3,
    num_classifiers=3,
    base_models=["microsoft/deberta-base", "roberta-base"],
    dropout=0.2,
    max_tokens=256,
    label_map=comp_id2label,
)

2024-06-20 22:12:41.867 | DEBUG    | triagerx.model.module_factory:get_model:54 - Instantiating model of class: <class 'triagerx.model.triagerx_dev_model.TriagerxDevModel'>
2024-06-20 22:12:41.869 | DEBUG    | triagerx.model.module_factory:get_model:65 - Including number of filters and classifiers
2024-06-20 22:12:41.870 | DEBUG    | triagerx.model.triagerx_dev_model:__init__:24 - Loading base transformer models: ['microsoft/deberta-base', 'roberta-base']
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
comp_model.load_state_dict(torch.load(component_model_weights))

<All keys matched successfully>

In [17]:
similarity_model = SentenceTransformer("all-mpnet-base-v2")

In [18]:
train_embeddings_path = "/home/mdafifal.mamun/notebooks/triagerX/data/openj9_embeddings/embeddings.npy"

In [19]:
encodings = similarity_model.encode(
    df_train.text.tolist(),
    show_progress_bar=True
)

import numpy as np

np.save(train_embeddings_path, encodings)

Batches:   0%|          | 0/83 [00:00<?, ?it/s]

# Grid Search

In [21]:
df_test = pd.read_csv("/home/mdafifal.mamun/notebooks/triagerX/data/openj9/last_contribution/openj9_test.csv")

In [22]:
def get_recommendation(trx, test_idx, k_comp, k_dev, k_rank, sim):
    test_data = df_test.iloc[test_idx]

    # print("Issue URL: ", test_data["issue_url"])
    # print("Actual owner: ", test_data["owner"])
    # print("Actual component: ", test_data["component"])

    return trx.get_recommendation(
        test_data.text,
        k_comp=k_comp,
        k_dev=k_dev,
        k_rank=k_rank,
        similarity_threshold=sim
    )

In [23]:
def get_topk_score(recommendations, top_k):
    combined_total = 0
    dl_total = 0
    sim_total = 0

    for idx in range(len(df_test)):
        actual = df_test.iloc[idx]["owner"]
        combined_recommended = recommendations[idx]["combined_ranking"][:top_k]
        dl_recommended = recommendations[idx]["predicted_developers"][:top_k]
        sim_recommended = recommendations[idx]["similar_devs"][:top_k]

        print("Actual: ", actual)
        print("DL: ", dl_recommended)

        if actual in combined_recommended:
            combined_total += 1

        if actual in dl_recommended:
            dl_total += 1

        if actual in sim_recommended:
            sim_total += 1

    return dl_total/len(df_test), sim_total/len(df_test), combined_total/len(df_test)
    

In [24]:
def evaluate_recommendations(params):
    # Extract parameters
    similarity_prediction_weight = params['similarity_prediction_weight']
    time_decay_factor = params['time_decay_factor']
    direct_assignment_score = params['direct_assignment_score']
    contribution_score = params['contribution_score']
    discussion_score = params['discussion_score']
    similarity_threshold = params['similarity_threshold']

    trx = TriagerX(
        component_prediction_model=comp_model,
        developer_prediction_model=dev_model,
        similarity_model=similarity_model,
        issues_path="/home/mdafifal.mamun/notebooks/triagerX/data/openj9/openj9_issue_data_6_7_24",
        train_embeddings=train_embeddings_path,
        developer_id_map=lbl2idx,
        component_id_map=comp_lbl2id,
        expected_developers=expected_users,
        train_data=df_train,
        device="cuda",
        similarity_prediction_weight=similarity_prediction_weight,
        time_decay_factor=time_decay_factor,
        direct_assignment_score=direct_assignment_score,
        contribution_score=contribution_score,
        discussion_score=discussion_score
    )

    recommendations = []

    for i in tqdm(range(len(df_test)), total=len(df_test), desc="Processing..."):
        rec = get_recommendation(trx, i, k_comp=3, k_dev=20, k_rank=20, sim=similarity_threshold)
        recommendations.append(rec)

    top_3 = get_topk_score(recommendations, 3)
    top_5 = get_topk_score(recommendations, 5)
    top_10 = get_topk_score(recommendations, 10)
    top_20 = get_topk_score(recommendations, 20)

    return top_3, top_5, top_10, top_20

In [36]:
parameter_ranges = {
    'similarity_prediction_weight': [0.6, 0.7, 0.8, 0.9],
    'time_decay_factor': [0.01, 0.05, 0.10],
    'direct_assignment_score': [1.0, 1.5, 2.0, 3.0],
    'contribution_score': [1.0, 1.5, 2.0, 3.0],
    'discussion_score': [0.5, 1.0, 1.5],
    'similarity_threshold': [0.5, 0.6, 0.7]
}

In [37]:
len(list(itertools.product(*parameter_ranges.values())))

1728

In [33]:
import itertools

total_combinations = len(list(itertools.product(*parameter_ranges.values())))

# Initialize an empty list to store results
results = []

index = 1
output_file = 'grid_search_results.csv'

# Iterate over all combinations
for params in itertools.product(*parameter_ranges.values()):
    
    print(f"Running Grid Search... {index}/{total_combinations}")
    index += 1
    
    params_dict = {
        'similarity_prediction_weight': params[0],
        'time_decay_factor': params[1],
        'direct_assignment_score': params[2],
        'contribution_score': params[3],
        'discussion_score': params[4],
        'similarity_threshold': params[5]
    }

    top_3, top_5, top_10, top_20 = evaluate_recommendations(params_dict)

    # Append results to the list
    results.append({
        'similarity_prediction_weight': params_dict['similarity_prediction_weight'],
        'time_decay_factor': params_dict['time_decay_factor'],
        'direct_assignment_score': params_dict['direct_assignment_score'],
        'contribution_score': params_dict['contribution_score'],
        'discussion_score': params_dict['discussion_score'],
        'similarity_threshold': params_dict['similarity_threshold'],
        'T3DL': top_3[0],
        'T3Sim': top_3[1],
        'T3Com': top_3[2],
        'T5DL': top_5[0],
        'T5Sim': top_5[1],
        'T5Com': top_5[2],
        'T10DL': top_10[0],
        'T10Sim': top_10[1],
        'T10Com': top_10[2],
        'T20DL': top_20[0],
        'T20Sim': top_20[1],
        'T20Com': top_20[2],
    })

    df = pd.DataFrame(results)
    df.to_csv(output_file, index=False)

    print(f"Grid search results saved to {output_file}")

df = pd.DataFrame(results)

# Write DataFrame to CSV file
df.to_csv(output_file, index=False)

print(f"Grid search results saved to {output_file}")

2024-06-20 22:15:14.914 | DEBUG    | __main__:__init__:55 - Using device: cuda
2024-06-20 22:15:14.916 | DEBUG    | __main__:__init__:56 - Loading embeddings for existing issues...


Running Grid Search... 1/5


Processing...:   0%|          | 0/379 [00:00<?, ?it/s]

2024-06-20 22:15:15.131 | WARNING  | __main__:_get_historical_contributors:289 - Skipped users: {'R-Santhir', 'zl-wang', 'dylanjtuttle', 'andrew-m-leonard', 'cathyzhyi', 'lumpfish', 'irinarada', 'M-Davies', 'manashaVetrivelu', 'andrewcraik', 'IBMJimmyk'} because they don't exist in the expected developers list
Processing...:   5%|▍         | 18/379 [00:02<00:42,  8.58it/s]2024-06-20 22:15:17.526 | WARNING  | __main__:_get_historical_contributors:289 - Skipped users: {'zl-wang', 'karianna', 'andrew-m-leonard', 'VermaSh', 'IBMJimmyk'} because they don't exist in the expected developers list
2024-06-20 22:15:17.682 | WARNING  | __main__:_get_historical_contributors:289 - Skipped users: {'vij-singh', 'renfeiw', 'mnalam-p', 'R-Santhir', 'zl-wang', 'github-actions[bot]', 'cathyzhyi', 'pelemie', 'Thihup', 'sophia-guo', 'OscarQQ', 'andrewcraik', 'AlexeyKhrabrov'} because they don't exist in the expected developers list
Processing...:  18%|█▊        | 70/379 [00:09<00:37,  8.29it/s]2024-06-20 2

Actual:  a7ehuo
DL:  ['mpirvu', 'hzongaro', 'dsouzai']
Actual:  dsouzai
DL:  ['JasonFengJ9', 'babsingh', 'llxia']
Actual:  JasonFengJ9
DL:  ['ymanton', 'BradleyWood', 'hangshao0']
Actual:  dsouzai
DL:  ['dsouzai', 'ymanton', 'dmitry-ten']
Actual:  a7ehuo
DL:  ['babsingh', 'pshipton', 'keithc-ca']
Actual:  keithc-ca
DL:  ['keithc-ca', 'pshipton', 'babsingh']
Actual:  JasonFengJ9
DL:  ['keithc-ca', 'pshipton', 'JasonFengJ9']
Actual:  LinHu2016
DL:  ['pshipton', 'mikezhang1234567890', 'hzongaro']
Actual:  JasonFengJ9
DL:  ['JasonFengJ9', 'babsingh', 'keithc-ca']
Actual:  babsingh
DL:  ['knn-k', 'pshipton', 'fjeremic']
Actual:  babsingh
DL:  ['ChengJin01', 'dchopra001', 'babsingh']
Actual:  JasonFengJ9
DL:  ['dsouzai', 'JasonFengJ9', 'jdmpapin']
Actual:  babsingh
DL:  ['babsingh', 'nbhuiyan', 'llxia']
Actual:  babsingh
DL:  ['babsingh', 'fengxue-IS', 'smlambert']
Actual:  theresa-m
DL:  ['keithc-ca', 'dsouzai', 'pshipton']
Actual:  mpirvu
DL:  ['dmitry-ten', 'thallium', 'harryyu1994']
Actu

...

Actual:  a7ehuo
DL:  ['mpirvu', 'hzongaro', 'dsouzai']
Actual:  dsouzai
DL:  ['JasonFengJ9', 'babsingh', 'llxia']
Actual:  JasonFengJ9
DL:  ['ymanton', 'BradleyWood', 'hangshao0']
Actual:  dsouzai
DL:  ['dsouzai', 'ymanton', 'dmitry-ten']
Actual:  a7ehuo
DL:  ['babsingh', 'pshipton', 'keithc-ca']
Actual:  keithc-ca
DL:  ['keithc-ca', 'pshipton', 'babsingh']
Actual:  JasonFengJ9
DL:  ['keithc-ca', 'pshipton', 'JasonFengJ9']
Actual:  LinHu2016
DL:  ['pshipton', 'mikezhang1234567890', 'hzongaro']
Actual:  JasonFengJ9
DL:  ['JasonFengJ9', 'babsingh', 'keithc-ca']
Actual:  babsingh
DL:  ['knn-k', 'pshipton', 'fjeremic']
Actual:  babsingh
DL:  ['ChengJin01', 'dchopra001', 'babsingh']
Actual:  JasonFengJ9
DL:  ['dsouzai', 'JasonFengJ9', 'jdmpapin']
Actual:  babsingh
DL:  ['babsingh', 'nbhuiyan', 'llxia']
Actual:  babsingh
DL:  ['babsingh', 'fengxue-IS', 'smlambert']
Actual:  theresa-m
DL:  ['keithc-ca', 'dsouzai', 'pshipton']
Actual:  mpirvu
DL:  ['dmitry-ten', 'thallium', 'harryyu1994']
Actu

Processing...:   3%|▎         | 11/379 [00:01<00:34, 10.71it/s]2024-06-20 22:17:34.035 | WARNING  | __main__:_get_historical_contributors:289 - Skipped users: {'ashu-mehra', 'cathyzhyi', 'IBMJimmyk', 'alexey-anufriev'} because they don't exist in the expected developers list
2024-06-20 22:17:34.170 | WARNING  | __main__:_get_historical_contributors:289 - Skipped users: {'karianna', 'sophia-guo', 'dipak-bagadiya'} because they don't exist in the expected developers list
Processing...:  17%|█▋        | 63/379 [00:06<00:31, 10.17it/s]2024-06-20 22:17:39.381 | WARNING  | __main__:_get_historical_contributors:289 - Skipped users: {'R-Santhir', 'andrewcraik', 'cathyzhyi', 'ThanHenderson', 'mayshukla', 'ashu-mehra'} because they don't exist in the expected developers list
2024-06-20 22:17:39.555 | WARNING  | __main__:_get_historical_contributors:289 - Skipped users: {'vij-singh', 'zl-wang', 'mateuszrzeszutek', 'lumpfish', 'klangman', 'VermaSh', 'EmanElsaban', 'Leonardo2718'} because they don'

Actual:  a7ehuo
DL:  ['mpirvu', 'hzongaro', 'dsouzai']
Actual:  dsouzai
DL:  ['JasonFengJ9', 'babsingh', 'llxia']
Actual:  JasonFengJ9
DL:  ['ymanton', 'BradleyWood', 'hangshao0']
Actual:  dsouzai
DL:  ['dsouzai', 'ymanton', 'dmitry-ten']
Actual:  a7ehuo
DL:  ['babsingh', 'pshipton', 'keithc-ca']
Actual:  keithc-ca
DL:  ['keithc-ca', 'pshipton', 'babsingh']
Actual:  JasonFengJ9
DL:  ['keithc-ca', 'pshipton', 'JasonFengJ9']
Actual:  LinHu2016
DL:  ['pshipton', 'mikezhang1234567890', 'hzongaro']
Actual:  JasonFengJ9
DL:  ['JasonFengJ9', 'babsingh', 'keithc-ca']
Actual:  babsingh
DL:  ['knn-k', 'pshipton', 'fjeremic']
Actual:  babsingh
DL:  ['ChengJin01', 'dchopra001', 'babsingh']
Actual:  JasonFengJ9
DL:  ['dsouzai', 'JasonFengJ9', 'jdmpapin']
Actual:  babsingh
DL:  ['babsingh', 'nbhuiyan', 'llxia']
Actual:  babsingh
DL:  ['babsingh', 'fengxue-IS', 'smlambert']
Actual:  theresa-m
DL:  ['keithc-ca', 'dsouzai', 'pshipton']
Actual:  mpirvu
DL:  ['dmitry-ten', 'thallium', 'harryyu1994']
Actu

Processing...: 100%|██████████| 379/379 [00:34<00:00, 10.98it/s]

Actual:  a7ehuo
DL:  ['mpirvu', 'hzongaro', 'dsouzai']
Actual:  dsouzai
DL:  ['JasonFengJ9', 'babsingh', 'llxia']
Actual:  JasonFengJ9
DL:  ['ymanton', 'BradleyWood', 'hangshao0']
Actual:  dsouzai
DL:  ['dsouzai', 'ymanton', 'dmitry-ten']
Actual:  a7ehuo
DL:  ['babsingh', 'pshipton', 'keithc-ca']
Actual:  keithc-ca
DL:  ['keithc-ca', 'pshipton', 'babsingh']
Actual:  JasonFengJ9
DL:  ['keithc-ca', 'pshipton', 'JasonFengJ9']
Actual:  LinHu2016
DL:  ['pshipton', 'mikezhang1234567890', 'hzongaro']
Actual:  JasonFengJ9
DL:  ['JasonFengJ9', 'babsingh', 'keithc-ca']
Actual:  babsingh
DL:  ['knn-k', 'pshipton', 'fjeremic']
Actual:  babsingh
DL:  ['ChengJin01', 'dchopra001', 'babsingh']
Actual:  JasonFengJ9
DL:  ['dsouzai', 'JasonFengJ9', 'jdmpapin']
Actual:  babsingh
DL:  ['babsingh', 'nbhuiyan', 'llxia']
Actual:  babsingh
DL:  ['babsingh', 'fengxue-IS', 'smlambert']
Actual:  theresa-m
DL:  ['keithc-ca', 'dsouzai', 'pshipton']
Actual:  mpirvu
DL:  ['dmitry-ten', 'thallium', 'harryyu1994']
Actu